## Import Dependencies

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# trl: Transformer Reinforcement Learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead, set_seed
from trl import create_reference_model

import torch
import evaluate

import numpy as np
import pandas as pd

# tqdm library makes the loops show a smart progress meter.
from tqdm import tqdm
tqdm.pandas()

## Load BioBart Model, Prepare Reward Model

In [2]:
model_name="./finetuned-checkpoints/biobart-base--mimic-cxr/checkpoint-19600"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(model, is_trainable=True)
ref_model = create_reference_model(ppo_model)

#from models.reward import get_reward_model
#reward_model = get_reward_model()

In [3]:
from radgraph import F1RadGraph

f1radgraph_reward = F1RadGraph(reward_level="partial")

## Build Dataset

In [4]:
from pathlib import Path
import datasets
from datasets import Image
from torchvision import transforms
import os.path

#dataset_config = 'mimic-cxr','mimic-iii'  
#split = 'train','validate',test
def build_dataset(dataset_config, tokenizer, split):
    def generate_image_path(line):
        return str(Path(data_path).joinpath(dataset_config).joinpath(line.strip().split(',')[0]))
    
    data_path = '/nfs/turbo/umms-vgvinodv/data/bioNLP23-Task-1B/data/'
    
    findings_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.findings.tok')
    impression_file_path = Path(data_path).joinpath(dataset_config).joinpath(split+'.impression.tok')

    findings = [line.strip() for line in open(findings_file_path).readlines()]
    impression = [line.strip() for line in open(impression_file_path).readlines()]
    
    dataset = datasets.Dataset.from_dict({"text":findings, "query":impression})   
    
    def tokenize(samples):
        input_text = [" ".join(['summarize:',text]) for text in samples["text"]]
        samples["input_ids"] = tokenizer(input_text)["input_ids"]
        return samples
    
    dataset = dataset.map(tokenize, batched=True, num_proc=4, remove_columns=["text"])
    
    dataset.set_format(type="torch")
    return dataset

dataset_config = "mimic-cxr"
tokenized_train_data = build_dataset(dataset_config,tokenizer,"train")

Map (num_proc=4):   0%|          | 0/125417 [00:00<?, ? examples/s]

In [5]:
print(tokenized_train_data)

Dataset({
    features: ['query', 'input_ids'],
    num_rows: 125417
})


In [6]:
print(tokenized_train_data[0])

{'query': 'No acute cardiopulmonary process.', 'input_ids': tensor([    0, 18581,  3916,  2072,    35,   345,    16,   117, 22628, 13581,
            6, 16415,  9799, 22089, 15727,    50, 45001,  6157,   368,  3631,
            4,   163, 16908, 12029,  8244,  5963,  1043,  2192,    14,   144,
          533,  3594, 42490, 21841,     4,    20,  1886,   118, 12781,   118,
         1988,  6204, 33585,    16,  2340,     4,  2893,  7418,   695,    81,
            5,   314, 10665,     6,  2905,   624,     5,  6181,     4,    20,
         4356,  4628,  2853, 28398,    16, 40618, 36774,     4, 38207, 42011,
         1571,     9,     5, 41834,   314,  2958,     8,  3821, 21443,    32,
         1581,     4,     2])}


In [7]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [8]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=8
batch_size=32

config = PPOConfig(
    model_name=model_name,    
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config, 
                         model=ppo_model, 
                         ref_model=ref_model, 
                         tokenizer=tokenizer, 
                         dataset=tokenized_train_data, 
                         data_collator=collator)

In [9]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "max_new_tokens": 512,
}

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    prompt_tensors = batch["input_ids"]
    #print(batch["input_ids"])
    #print(batch["query"])
    #break

    # Get response from FLAN-T5/PEFT LLM.
    summary_tensors = []

    for prompt_tensor in prompt_tensors:        
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)        
        summary_tensors.append(summary.squeeze())
    
    # This needs to be called "response".
    batch["response"] = [tokenizer.decode(r.squeeze(),skip_special_tokens=True) for r in summary_tensors]
    
    #print(batch["response"])
    #break
    
    # Compute reward outputs.    
    rewards = f1radgraph_reward(hyps=batch["response"], refs=batch["query"])[1]
    reward_tensors = [torch.tensor(reward) for reward in rewards] 
    
    #print(reward_tensors)
    #break

    # Run PPO step.
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)
    
    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
1it [00:08,  8.30s/it]

objective/kl: 0.0
ppo/returns/mean: 0.5267552137374878
ppo/policy/advantages_mean: 0.0150148905813694
---------------------------------------------------------------------------------------------------


2it [00:14,  7.36s/it]

objective/kl: -0.0616590790450573
ppo/returns/mean: 0.5280265808105469
ppo/policy/advantages_mean: -0.02006658911705017
---------------------------------------------------------------------------------------------------


3it [00:24,  8.48s/it]

...

objective/kl: 4.576208591461182
ppo/returns/mean: -0.05052005499601364
ppo/policy/advantages_mean: 0.0
---------------------------------------------------------------------------------------------------


3918it [4:24:18,  3.21s/it]

objective/kl: 3.958761692047119
ppo/returns/mean: -0.040133409202098846
ppo/policy/advantages_mean: -1.6763806343078613e-08
---------------------------------------------------------------------------------------------------


3919it [4:24:22,  4.05s/it]

objective/kl: 2.7029216289520264
ppo/returns/mean: -0.023888345807790756
ppo/policy/advantages_mean: -9.313225746154785e-09
---------------------------------------------------------------------------------------------------


In [12]:
#### get a batch from the dataset
bs = 32
game_data = dict()
tokenized_train_data.set_format("pandas")
df_batch = tokenized_train_data[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    output = ref_model.generate(torch.tensor(query_tensors[i]).unsqueeze(dim=0).to("cuda"), **generation_kwargs).squeeze()
    response_tensors_ref.append(output)
    output = ppo_model.generate(torch.tensor(query_tensors[i]).unsqueeze(dim=0).to("cuda"), **generation_kwargs).squeeze()
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i],skip_special_tokens=True) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i],skip_special_tokens=True) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
game_data["rewards (before)"] = f1radgraph_reward(hyps=game_data["response (before)"], refs=game_data["query"])[1]

game_data["rewards (after)"] = f1radgraph_reward(hyps=game_data["response (after)"], refs=game_data["query"])[1]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

/home/varu/.local/lib/python3.10/site-packages/radgraph/dygie/data/dataset_readers/dygie.py:192: UserWarning: Document 0 has a sentence with a single token or no tokens. This may break the modeling code.
  warnings.warn(msg)


,query,response (before),response (after),rewards (before),rewards (after)
0,No acute cardiopulmonary abnormality.,No acute cardiopulmonary abnormality.,--1-,1.000000,0.0
1,"Right greater than left perihilar opacities, r...","Bilateral perihilar airspace opacities, right ...",pneum pneum pneum-,0.571429,0.0
2,No acute cardiopulmonary process.,No acute cardiopulmonary process.,--- pneum,1.000000,0.0
3,"Moderate-sized right pleural effusion, which a...","Moderate-sized right pleural effusion, slightl...",pneum pneum pneum-,0.818182,0.0
4,Interval resolution of wedge-shaped opacity at...,Interval resolution of wedge-shaped opacity at...,pneum pneum pneum-,1.000000,0.0
5,"Improving left basilar pneumonia. However, it ...","Left basilar pneumonia has improved, but not c...",pneum pneum pneum-,0.266667,0.0
6,No acute intrathoracic process.,No acute intrathoracic process.,--- pneum,1.000000,0.0
7,1. Faint opacity at the left base likely refle...,1. Ill-defined opacity at the left base on the...,pneum pneum pneum-,0.516129,0.0
8,No evidence of acute cardiopulmonary process. ...,Right IJ central venous line with the tip term...,pneum pneum pneum-,0.380952,0.0
9,No acute cardiopulmonary abnormality.,No acute cardiopulmonary abnormality.,--- pneum,1.000000,0.0
